In [1]:
def plot_this(s,title=''):
    """

    """
    import pylab
    s = s.squeeze()
    if s.ndim ==1:
        pylab.plot(s)
    else:
        pylab.imshow(s,aspect='auto')
        pylab.title(title)
    pylab.show()

import torch 
def calculate_scaling_factor(clean_audio, noise_audio, target_snr):
    """Calculate the scaling factor to adjust noise to the target SNR level."""
    target_snr = float(target_snr)
    clean_power = torch.mean(clean_audio**2)
    noise_power = torch.mean(noise_audio**2)
    desired_noise_power = clean_power / (10 ** (target_snr / 10))
    scaling_factor = torch.sqrt(desired_noise_power / noise_power)
    return scaling_factor

In [2]:
# import os
# import re
# import shutil
# import random
# from collections import defaultdict
# from pathlib import Path

# def parse_reader_id(filename):
#     """Extract reader ID from filename like: book_0000_chp_009_reader_84372_2.wav"""
#     match = re.search(r"reader_(\d+)_", filename)
#     return match.group(1) if match else None

# def organize_clips_from_flat_dir(src_dir, dest_dir, num_speakers=10, clips_per_speaker=10, seed=42):
#     random.seed(seed)
#     src_dir = Path(src_dir)
#     dest_dir = Path(dest_dir)
#     clean_wav_dir = dest_dir / "clean_wav"
#     clean_train_dir = dest_dir / "clean_train"
#     clean_wav_dir.mkdir(parents=True, exist_ok=True)
#     clean_train_dir.mkdir(parents=True, exist_ok=True)

#     # Step 1: Group files by speaker (reader)
#     speaker_to_files = defaultdict(list)
#     for file in src_dir.glob("*.wav"):
#         reader_id = parse_reader_id(file.name)
#         if reader_id:
#             speaker_to_files[reader_id].append(file)

#     # Step 2: Filter and randomly select speakers
#     eligible_speakers = [s for s, files in speaker_to_files.items() if len(files) >= clips_per_speaker]
#     selected_speakers = random.sample(eligible_speakers, min(num_speakers, len(eligible_speakers)))

#     for reader_id in selected_speakers:
#         files = random.sample(speaker_to_files[reader_id], clips_per_speaker)
#         main_clip = files[0]
#         support_clips = files[1:]

#         # Copy main clip to clean_wav/
#         shutil.copy(main_clip, clean_wav_dir / main_clip.name)

#         # Copy other 9 to clean_train/<main_clip_stem>/
#         support_dir = clean_train_dir / main_clip.stem
#         support_dir.mkdir(parents=True, exist_ok=True)

#         for clip in support_clips:
#             shutil.copy(clip, support_dir / clip.name)

#         print(f"Processed speaker {reader_id}: main={main_clip.name}, train={len(support_clips)} clips")

# # Example usage
# organize_clips_from_flat_dir(
#     src_dir="/data/ephraim/datasets/DNS-Challenge_old/datasets/clean",
#     dest_dir="/data/ephraim/datasets/known_noise/undiff_exps2/exp_p5lib_net2b/"
# )



In [9]:
import os
import re
import shutil
import random
from collections import defaultdict
from pathlib import Path
import torchaudio

def parse_reader_id(filename):
    match = re.search(r"reader_(\d+)_", filename)
    return match.group(1) if match else None

def organize_and_resample_clips(src_dir, dest_dir, num_speakers=10, clips_per_speaker=10, seed=42, target_sr=16000):
    random.seed(seed)
    src_dir = Path(src_dir)
    dest_dir = Path(dest_dir)

    original_clean_wav_dir = dest_dir / "original_clean_wav"
    original_clean_train_dir = dest_dir / "original_clean_train"
    clean_wav_dir = dest_dir / "clean_wav"
    clean_train_dir = dest_dir / "clean_train"

    for d in [original_clean_wav_dir, original_clean_train_dir, clean_wav_dir, clean_train_dir]:
        d.mkdir(parents=True, exist_ok=True)

    speaker_to_files = defaultdict(list)
    for file in src_dir.glob("*.wav"):
        reader_id = parse_reader_id(file.name)
        if reader_id:
            speaker_to_files[reader_id].append(file)

    eligible_speakers = [s for s, files in speaker_to_files.items() if len(files) >= clips_per_speaker]
    selected_speakers = random.sample(eligible_speakers, min(num_speakers, len(eligible_speakers)))

    for idx, reader_id in enumerate(selected_speakers):
        files = random.sample(speaker_to_files[reader_id], clips_per_speaker)
        main_clip = files[0]
        support_clips = files[1:]
        file_id = f"fileid_{idx}"

        # Original
        shutil.copy(main_clip, original_clean_wav_dir / main_clip.name)
        orig_train_dir = original_clean_train_dir / file_id
        orig_train_dir.mkdir(parents=True, exist_ok=True)
        for clip in support_clips:
            shutil.copy(clip, orig_train_dir / clip.name)

        # Resampled
        waveform, sr = torchaudio.load(main_clip)
        if sr != target_sr:
            waveform = torchaudio.functional.resample(waveform, sr, target_sr)
        torchaudio.save(clean_wav_dir / f"{main_clip.stem}_{file_id}.wav", waveform, target_sr,encoding="PCM_F")

        resampled_dir = clean_train_dir / file_id
        resampled_dir.mkdir(parents=True, exist_ok=True)
        for clip in support_clips:
            waveform, sr = torchaudio.load(clip)
            if sr != target_sr:
                waveform = torchaudio.functional.resample(waveform, sr, target_sr)
            torchaudio.save(resampled_dir / clip.name, waveform, target_sr,encoding="PCM_F")

        print(f"Saved: speaker {reader_id} as {file_id}")

# Example:
src_dir="/data/ephraim/datasets/DNS-Challenge_old/datasets/clean"
dest_dir="/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/"
organize_and_resample_clips(src_dir, dest_dir, num_speakers=20, clips_per_speaker=10)
    

Saved: speaker 03714 as fileid_0
Saved: speaker 09440 as fileid_1
Saved: speaker 11377 as fileid_2
Saved: speaker 04199 as fileid_3
Saved: speaker 11207 as fileid_4
Saved: speaker 06475 as fileid_5
Saved: speaker 04433 as fileid_6
Saved: speaker 00123 as fileid_7
Saved: speaker 06271 as fileid_8
Saved: speaker 04278 as fileid_9
Saved: speaker 07893 as fileid_10
Saved: speaker 08200 as fileid_11
Saved: speaker 06967 as fileid_12
Saved: speaker 02964 as fileid_13
Saved: speaker 05299 as fileid_14
Saved: speaker 11329 as fileid_15
Saved: speaker 03259 as fileid_16
Saved: speaker 02584 as fileid_17
Saved: speaker 00727 as fileid_18
Saved: speaker 10008 as fileid_19


In [10]:
import shutil
from pathlib import Path

def copy_directory(src_dir, dest_dir, overwrite=False):
    """
    Copy a directory and all its contents from src_dir to dest_dir.

    Parameters:
    - src_dir (str or Path): Source directory path
    - dest_dir (str or Path): Destination directory path
    - overwrite (bool): If True, will overwrite the destination if it exists
    """
    src_dir = Path(src_dir)
    dest_dir = Path(dest_dir)

    if not src_dir.exists():
        raise FileNotFoundError(f"Source directory '{src_dir}' does not exist.")

    if dest_dir.exists():
        if overwrite:
            shutil.rmtree(dest_dir)
        else:
            raise FileExistsError(f"Destination '{dest_dir}' already exists. Use overwrite=True to replace it.")

    shutil.copytree(src_dir, dest_dir)
    print(f"Copied '{src_dir}' → '{dest_dir}'")

# Example usage:


In [11]:
# copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_libr20/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_libr16/", overwrite=False)


In [12]:
import os
from pathlib import Path

def insert_snr_tag_in_filenames(directory, tag="snr5"):
    """
    Insert '_snr5' before '_fileid_' in all .wav filenames within `directory`.
    """
    directory = Path(directory)
    for wav_file in directory.glob("*.wav"):
        old_name = wav_file.name
        if "_fileid_" in old_name:
            parts = old_name.split("_fileid_")
            new_name = f"{parts[0]}_{tag}_fileid_{parts[1]}"
            new_path = wav_file.with_name(new_name)
            os.rename(wav_file, new_path)
            print(f"Renamed: {old_name} → {new_name}")
        else:
            print(f"Skipped (no _fileid_): {old_name}")

# Example usage:


bbc dataset

#bbc2

In [13]:
import random, csv, shutil, torch, torchaudio, numpy as np
from pathlib import Path

# ------------------------------------------------------------------
# CONFIG ------------------------------------------------------------------
TARGET_SR  = 16_000          # Hz
TARGET_SNR = 5.0             # dB
SKIP_SEC   = 2               # do not use first 2 s of noise
SEED       = 123
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ------------------------------------------------------------------
# UTILITIES ---------------------------------------------------------------
def resample_mono(w, old_sr, new_sr=TARGET_SR):
    w = w[:1]                                   # first channel
    return torchaudio.functional.resample(w, old_sr, new_sr) if old_sr != new_sr else w

def scale_to_snr(clean, noise, snr_db):
    return noise * torch.sqrt(torch.mean(clean**2) /
                              (torch.mean(noise**2) * 10**(snr_db/10)))

def scan_noise(root):
    """Return shuffled list of all noise paths."""
    paths = list(Path(root).rglob("*.wav"))
    random.shuffle(paths)
    return paths

# ------------------------------------------------------------------
# MAIN --------------------------------------------------------------------
def mix_clean_with_long_noise(dest_dir, noise_root, snr_db=TARGET_SNR):
    dest = Path(dest_dir)
    clean_dir = dest / "clean_wav"
    out = {k: dest/k for k in ("noisy_wav","noises","original_noises")}
    for d in out.values():
        d.mkdir(parents=True, exist_ok=True)

    mapping_rows = []
    skip_frames  = SKIP_SEC * TARGET_SR
    noise_pool   = scan_noise(noise_root)
    noise_idx    = 0                            # pointer into shuffled pool

    for clean_file in sorted(clean_dir.glob("*.wav")):
        # -------- load clean (mono 16 k) ----------------------------------
        c, sr = torchaudio.load(clean_file)
        c = resample_mono(c, sr)
        L = c.shape[-1]

        # -------- find a unique noise WAV long enough ---------------------
        while noise_idx < len(noise_pool):
            n_path = noise_pool[noise_idx]; noise_idx += 1
            n_raw, sr_n = torchaudio.load(n_path)
            n_mono      = resample_mono(n_raw, sr_n)[:, skip_frames:]
            if n_mono.shape[-1] >= L:              # usable!
                break
        else:
            raise RuntimeError("Ran out of long noise files.")

        start = random.randint(0, n_mono.shape[-1] - L)
        seg   = n_mono[:, start:start+L]
        seg   = scale_to_snr(c, seg, snr_db)
        noisy = c + seg

        # -------- save outputs -------------------------------------------
        shutil.copy(n_path, out["original_noises"]/clean_file.name)
        torchaudio.save(out["noises"]/clean_file.name, seg,   TARGET_SR)
        torchaudio.save(out["noisy_wav"]/clean_file.name, noisy, TARGET_SR)

        mapping_rows.append([
            str(clean_file),
            str(out["noisy_wav"]/clean_file.name),
            str(out["noises"]/clean_file.name),
            str(n_path),
            start + skip_frames,
            start + L - 1 + skip_frames,
            snr_db
        ])

        print(f"{clean_file.name:<25}  ←  {n_path.name}  |  SNR={snr_db} dB")

    # -------- write mapping CSV ------------------------------------------
    map_csv = dest / "noise_mapping.csv"
    with open(map_csv, "w", newline="") as f:
        csv.writer(f).writerows([["clean","noisy","noise_seg",
                                  "noise_src","start","end","snr_db"],
                                 *mapping_rows])
    print(f"\nMapping saved to {map_csv}")

# ------------------------------------------------------------------
# Example call
# ------------------------------------------------------------------
# mix_clean_with_long_noise(
#     dest_dir   = "/path/to/dest",
#     noise_root = "/path/to/BBC_noise_dataset",
#     snr_db     = 5.0)



In [14]:
mix_clean_with_long_noise(
    dest_dir  = "/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5",
    noise_root = "/data/ephraim/datasets/known_noise/bbc-noises/bbc20/",
    snr_db= 5.0)

book_00334_chp_0040_reader_11377_3_fileid_2.wav  ←  NHU05104248.wav  |  SNR=5.0 dB
book_00375_chp_0010_reader_02584_13_fileid_17.wav  ←  07044002.wav  |  SNR=5.0 dB
book_01265_chp_0062_reader_02964_24_fileid_13.wav  ←  NHU05086083.wav  |  SNR=5.0 dB
book_01296_chp_0015_reader_00727_63_fileid_18.wav  ←  NHU05084027.wav  |  SNR=5.0 dB
book_01436_chp_0016_reader_04278_31_fileid_9.wav  ←  07015100.wav  |  SNR=5.0 dB
book_01652_chp_0035_reader_09440_9_fileid_1.wav  ←  07022168.wav  |  SNR=5.0 dB
book_02132_chp_0098_reader_00123_6_fileid_7.wav  ←  NHU05101064.wav  |  SNR=5.0 dB
book_03136_chp_0008_reader_10008_3_fileid_19.wav  ←  NHU05070103.wav  |  SNR=5.0 dB
book_03171_chp_0011_reader_04433_25_fileid_6.wav  ←  NHU05071088.wav  |  SNR=5.0 dB
book_03465_chp_0008_reader_06475_9_fileid_5.wav  ←  NHU05011211.wav  |  SNR=5.0 dB
book_03855_chp_0007_reader_11207_2_fileid_4.wav  ←  07015058.wav  |  SNR=5.0 dB
book_06051_chp_0008_reader_11329_7_fileid_15.wav  ←  07064025.wav  |  SNR=5.0 dB
book_0783

In [15]:
insert_snr_tag_in_filenames("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/noisy_wav", tag="snr5")

Renamed: book_00334_chp_0040_reader_11377_3_fileid_2.wav → book_00334_chp_0040_reader_11377_3_snr5_fileid_2.wav
Renamed: book_00375_chp_0010_reader_02584_13_fileid_17.wav → book_00375_chp_0010_reader_02584_13_snr5_fileid_17.wav
Renamed: book_01265_chp_0062_reader_02964_24_fileid_13.wav → book_01265_chp_0062_reader_02964_24_snr5_fileid_13.wav
Renamed: book_01296_chp_0015_reader_00727_63_fileid_18.wav → book_01296_chp_0015_reader_00727_63_snr5_fileid_18.wav
Renamed: book_01436_chp_0016_reader_04278_31_fileid_9.wav → book_01436_chp_0016_reader_04278_31_snr5_fileid_9.wav
Renamed: book_01652_chp_0035_reader_09440_9_fileid_1.wav → book_01652_chp_0035_reader_09440_9_snr5_fileid_1.wav
Renamed: book_02132_chp_0098_reader_00123_6_fileid_7.wav → book_02132_chp_0098_reader_00123_6_snr5_fileid_7.wav
Renamed: book_03136_chp_0008_reader_10008_3_fileid_19.wav → book_03136_chp_0008_reader_10008_3_snr5_fileid_19.wav
Renamed: book_03171_chp_0011_reader_04433_25_fileid_6.wav → book_03171_chp_0011_reader_0

In [ ]:
copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR-5/", overwrite=False)
copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR0/", overwrite=False)
copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR10/", overwrite=False)
copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net30_SNR5/", overwrite=False)
copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net40_SNR5/", overwrite=False)
copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_libr20_p_net2_6_SNR5/", overwrite=False)
copy_directory("/data/ephraim/datasets/known_noise/undiff_exps3/exp_librBBC20_net3_6_SNR5/", "/data/ephraim/datasets/known_noise/undiff_exps3/exp_libr20_p_net3_6_SNR5/", overwrite=False)
